# Config

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import json
from requests_html import HTMLSession, AsyncHTMLSession
import time
from datetime import datetime

In [2]:
# Set Variables
date_string = datetime.today().strftime('%Y%m%d')
weekStart = 1
weekEnd = 18

# Get IDs

In [3]:
# Read dictionary of IDs
ids = pd.read_csv('db/id_dict.csv', dtype={'player_id':'str'})
ids = ids[['full_name', 'player_id', 'id_sharks', 'id_ourlads']]

# Get Sleeper

In [4]:
def processPlayers():
    # Use requests_html to get players since sleeper-py seems to be broken here
    session = HTMLSession()
    r = session.get('https://api.sleeper.app/v1/players/nfl')
    df = pd.DataFrame(json.loads(r.html.html)).T
    df = df[[
        'player_id', 'full_name','birth_date','age','weight', 'height',
        'team','position','fantasy_positions','depth_chart_order','depth_chart_position',
        'status', 'injury_status','injury_notes','injury_start_date', 'practice_participation', 'practice_description',
        'years_exp',
        'active',
    ]]
    df = df.loc[df['position'].isin(['QB','RB','WR','TE','K','P','DEF'])]
    df.loc[df['position'].isin(['P', 'K']), 'position'] = 'PK'
    df.loc[df['position']=='DEF', 'full_name'] = df.loc[df['position']=='DEF', 'team']
    return df

# Get info on all players
players = processPlayers()

# Get Fantasy Sharks

In [5]:
# Get Fantasy Sharks predictions
session = AsyncHTMLSession()
sharks = pd.DataFrame()

# Loop through weeks and positions
posList = ['1','2','4','5','7','15','6']
weekList = [str(int(x)) for x in np.linspace(787, 804, 18)]
for posKey in posList:
    for weekKey in weekList[(weekStart-1):weekEnd]:
        url = f'https://www.fantasysharks.com/apps/bert/forecasts/projections.php?League=&Position={posKey}&scoring=2&Segment={weekKey}&uid=4'
        # Scrape
        r = await session.get(url)
        await r.html.arender()
        element = r.html.find('#toolData')[0].html
        # Convert to df
        df = pd.read_html(element)[0]
        df = df.loc[df['#'].str.isnumeric()]
        df['Week'] = int(weekKey) - 786
        sharks = pd.concat([sharks, df], axis=0, ignore_index=True)
        time.sleep(5)
        
# Convert raw prediction values to numeric
colList = ['Att', 'Comp', 'Pass Yds', 'Pass TDs', 'Int', 'Sck', 'Rush', 'Rsh Yds', 'Rsh TDs', 'Fum', 'Tgt', \
    'RZ Tgt', 'Rec', 'Rec Yds', 'Rec TDs', 'Kick Ret Yds', 'XPM', 'XPA', 'FGM', 'FGA', '10-19 FGM', '20-29 FGM', \
        '30-39 FGM', '40-49 FGM', '50+ FGM', 'Miss', 'Yds Allowed', 'Pts Agn', 'Scks', 'DefTD', 'Safts']
for col in colList:
    sharks[col] = sharks[col].astype('float64')
    
# Clean up
sharks = sharks.rename(columns={'Week':'week'})
# Create ID: Remove Rookie "R": format first name last name
cond1 = sharks['Player'].str[-1]=="R"
sharks.loc[cond1, 'Player'] = sharks.loc[cond1, 'Player'].str[:-1]
sharks['lName'] = sharks['Player'].str.split(", ", expand=True)[0]
sharks['fName'] = sharks['Player'].str.split(", ", expand=True)[1]
sharks['id_sharks'] = sharks['fName']+" "+sharks['lName']
sharks['id_sharks'] = sharks['id_sharks'].str.replace(".", "")
sharks['id_sharks'] = sharks['id_sharks'].str.title()
sharks['id_sharks'] = [" ".join(x.split(" ")[:2]) for x in sharks['id_sharks']]
# Reduce
sharks = sharks[['id_sharks', 'week']+colList]


# Get Ourlads

In [6]:
# Get OurLads data about punt/kick returners
session = AsyncHTMLSession()

# Get page
r = await session.get("https://www.ourlads.com/nfldepthcharts/depthcharts.aspx")
await r.html.arender()

element = r.html.find('#ctl00_phContent_gvChart')[0].html

# Convert to dataframe
ourlads = pd.read_html(element)[0]
# Filter for only the needed columns
ourlads = ourlads[['Team', 'Pos', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5']]
# Rename columns of Position Ranks; limit number of ranks to three
ourlads = ourlads.rename(columns={
    'Player 1':'1',
    'Player 2':'2',
    'Player 3':'3',
    'Player 4':'3',
    'Player 5':'3',
})
# Filter only relevant positions
posList = ['LWR', 'RWR', 'SWR', 'TE', 'QB', 'RB', 'PK', 'PR', 'KR', 'RES']
ourlads = ourlads.loc[ourlads['Pos'].isin(posList)]

# Transpose columns to rows to get position ranks in row form rather than column
ourlads = ourlads.melt(id_vars=["Team", "Pos"], 
    var_name="posRank", 
    value_name="playerName")
    # Rename columns to match MyFantasyLeague
ourlads = ourlads.rename(columns={'Team':'team', 'Pos':'pos'})
# Create id_ourlads column
ourlads = ourlads.dropna(subset='playerName')
ourlads['lName'] = ourlads['playerName'].str.split(", ", expand=True)[0]
ourlads['fName'] = ourlads['playerName'].str.split(", ", expand=True)[1].str.split(" ", expand=True)[0]
ourlads['id_ourlads'] = ourlads['fName'] + " " + ourlads['lName']
ourlads['id_ourlads'] = ourlads['id_ourlads'].str.replace(".", "")
ourlads['id_ourlads'] = ourlads['id_ourlads'].str.title()
ourlads['id_ourlads'] = [" ".join(x.split(" ")[:2]) for x in ourlads['id_ourlads']]

# Highlight PR & KRs
cond1 = ourlads['pos']=='PR'
cond2 = ourlads['pos']=='KR'
cond3 = ourlads['posRank']=="1"
prs = ourlads.loc[cond1 & cond3]['id_ourlads'].unique()
krs = ourlads.loc[cond2 & cond3]['id_ourlads'].unique()
ourlads.loc[ourlads['id_ourlads'].isin(prs), 'PR'] = True
ourlads.loc[ourlads['id_ourlads'].isin(krs), 'KR'] = True

# Reduce
ourlads = ourlads.loc[(ourlads['PR']==True) | (ourlads['KR']==True)]
ourlads = ourlads.drop_duplicates(subset='id_ourlads', ignore_index=True)
ourlads = ourlads[['id_ourlads', 'PR', 'KR']]

# Create predictions dataset

In [7]:
# Create predictions dataset
preds = players \
    .merge(ids[['player_id', 'id_sharks', 'id_ourlads']], how='left', on='player_id') \
    .merge(sharks, how='left', on='id_sharks') \
    .merge(ourlads, how='left', on='id_ourlads')
preds['date_prediction'] = date_string

# Flag any players who need to be added to the id_dict

In [8]:
# Flag previously unknown players
cond1 = ~players['player_id'].isin(ids['player_id'])
cond2 = players['full_name']!="Player Invalid"
unknowns = players.loc[cond1 & cond2]
print(len(unknowns))
if len(unknowns)>0:
    unknowns.to_csv('db/unknowns.csv', index=False)

3


# Update database

In [9]:
# Read old data
preds_db = pd.read_csv('db/records_preds.csv')
# Update
preds_db = preds_db.loc[preds_db['date_prediction']!=date_string]
preds_db = pd.concat([preds_db, preds], axis=0, ignore_index=False)
# Write new data
preds_db.to_csv('db/records_preds.csv', index=False)